In [3]:
import pandas as pd

# ----------------------------
# Step 1: Load CSVs
# ----------------------------
df_yf = pd.read_csv(r"C:\Users\devin\Desktop\ASSET ARBITRAGE\Gold_Futures_2004_2025.csv")
df_kaggle = pd.read_csv(r"C:\Users\devin\Desktop\ASSET ARBITRAGE\gold_kaggle_cleaned.csv")

# ----------------------------
# Step 2: Ensure Date column exists
# ----------------------------
# Check first column name
print("Yahoo columns:", df_yf.columns)

# Agar first column Date nahi hai, use rename
if df_yf.columns[0].lower() not in ['date']:
    df_yf.rename(columns={df_yf.columns[0]: 'Date'}, inplace=True)

# Remove duplicate columns if any
df_yf = df_yf.loc[:, ~df_yf.columns.duplicated()]

# ----------------------------
# Step 3: Convert Date to datetime
# ----------------------------
df_yf['Date'] = pd.to_datetime(df_yf['Date'], errors='coerce')
df_kaggle['Date'] = pd.to_datetime(df_kaggle['Date'], errors='coerce')

# Drop rows with invalid dates
df_yf = df_yf.dropna(subset=['Date'])
df_kaggle = df_kaggle.dropna(subset=['Date'])

# ----------------------------
# Step 4: Numeric columns fix
# ----------------------------
for col in ['Close','High','Low','Open','Volume']:
    if col in df_yf.columns:
        df_yf[col] = pd.to_numeric(df_yf[col], errors='coerce')

# Fill Price column from Close if Price not exist
if 'Price' not in df_yf.columns or df_yf['Price'].isnull().all():
    df_yf['Price'] = df_yf['Close']

# ----------------------------
# Step 5: Merge datasets
# ----------------------------
final = pd.concat([df_yf, df_kaggle]) \
          .drop_duplicates(subset=['Date']) \
          .sort_values('Date') \
          .reset_index(drop=True)

# Keep only date part
final['Date'] = final['Date'].dt.date

# ----------------------------
# Step 6: Save final CSV
# ----------------------------
final.to_csv(r"C:\Users\devin\Desktop\ASSET ARBITRAGE\final_gold_dataset.csv", index=False)

print("✅ Final dataset ready! Shape:", final.shape)
print(final.head())


Yahoo columns: Index(['Price', 'Close', 'High', 'Low', 'Open', 'Volume'], dtype='object')


C:\Users\devin\AppData\Local\Temp\ipykernel_6452\1170655749.py:25: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_yf['Date'] = pd.to_datetime(df_yf['Date'], errors='coerce')


✅ Final dataset ready! Shape: (6191, 7)
         Date       Close        High         Low        Open  Volume  \
0  2004-01-03  399.299988  396.399994  396.399994  396.399994     2.0   
1  2004-01-04  427.799988  432.000000  424.500000  427.200012  1561.0   
2  2004-01-06  394.600006  399.000000  393.299988  393.899994  1864.0   
3  2004-01-07  396.000000  396.000000  396.000000  396.000000   146.0   
4  2004-01-09  408.799988  407.299988  407.299988  407.299988     0.0   

        Price  
0  399.299988  
1  427.799988  
2  394.600006  
3  396.000000  
4  408.799988  
